In [1]:
# Connect with the thymio
from tdmclient import ClientAsync
from global_navigation import GlobalNavigation

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 6ca1ca45-4524-41eb-8cc4-fb023f64b9d0

In [2]:
await node.unlock()

In [2]:
from tdmclient import ClientAsync
from global_navigation import GlobalNavigation

# client = ClientAsync()
# node = await client.wait_for_node()
# await node.lock()

from Vision import Vision, show_many_img
import time
from motion_control import MotionControl
from robot_api import Thymio
from global_navigation import GlobalNavigation
import numpy as np
from kalman_filter import KalmanFilter

In [3]:
#create vision module object
visio = Vision()
visio.begin()

"""Point d'entrée principal"""
global_nav = GlobalNavigation()
Thymio = Thymio(node, client)
motion_control = MotionControl(Thymio)
kalman_filter =KalmanFilter()

Caméra initialisée sur le port 4


No path
No path


No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path


In [1]:
motion_control.stop_motor()

NameError: name 'motion_control' is not defined

In [4]:
thymio, goal, obstacles = [],[],[]

while not (thymio and goal and obstacles):
    _thymio, _goal, _obstacles = visio.get_thymio_goal_and_obstacles()
    time.sleep(0.2)
    if _thymio : 
        kalman_filter.kal_state = np.array(_thymio).reshape(3, 1)
    if _goal :
        goal = _goal
    if _obstacles :
        obstacles = _obstacles

print("everything ok to start")

kalman_filter.pre_variance = np.array([[0.01, 0, 0],
                                      [0, 0.01, 0],
                                      [0, 0, 0.01]])

# Path planning
path, _, _ = global_nav.dijkstra(thymio, goal, obstacles)
visio.camera_feed.vision_module.path = path.copy()
path.pop(0)
goal_point = path[0]
global_nav.plot_path(path, obstacles)

No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path


KeyboardInterrupt: 

No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path
No path


In [ ]:
try:
    while True:
        motion_control.obstacle_avoidance()
        if motion_control.path_tracking(motion_control.kal_state, goal_point):
           print("reached to goal")
           path.pop(0)
           if not path:
               motion_control.stop_motor()
               break
           else:
               goal_point = path[0]
        thymio,  *_ = visio.get_thymio_goal_and_obstacles()
        thymio = kalman_filter.check_camera(thymio)
        vl_displacement, vr_displacement = motion_control.get_displacement()
        kal_state, kal_variance = kalman_filter.estimate(vl_displacement, vr_displacement, thymio)
        visio.camera_feed.past_kalman_estimates.append(kal_state[:2])
except KeyboardInterrupt:
    print("Stop the program")
finally:
    visio.stop()
    print("Program finishes")